In [2]:
!pip install statsmodels

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stat
import statsmodels.api as sm
import os

from mpl_toolkits import mplot3d

pd.set_option('display.max_columns', 9999)

%matplotlib notebook

In [17]:
os.chdir('/work/')
med = pd.read_csv('eicu-collaborative-research-database-2.0/medication.csv.gz')

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def opioids (x):
    
    opiods = ['hydromorphone','hydrocodone','morphine','oxycodone','oxymorphone','codeine','fentanyl','meperidine',
          'tramadol','carfentanil','percocet','norco']

    for i in opiods:
        if i in str(x).lower():
            return 1
        
    return 0

In [6]:
def nsaids (x):
    
    non_op = ['acetaminophen','amitriptyline', 'doxepin', 'imipramine', 'desipramine', 'nortriptyline',
         'ibuprofen', 'naproxen', 'diclofenac', 'piroxicam', 'sulindac', 'indomethacin', 'ketorolac', 'meloxicam', 
          'celecoxib', 'ketoprofen', 'oxaprozin', 'toradol', 'valdecoxib', 'bextra', 'rofecoxib', 'vioxx', 'gabapentin', 
          'neurontin', 'cyclobenzaprine', 'duloxetine', 'cymbalta', 'pregabalin', 'lyrica', 'venlafaxine', 'effexor', 
          'tylenol', 'voltaren', 'naprosyn', 'paracetamol', 'aspirin']
    
    for j in non_op:
        if j in str(x).lower():
            return 1

    return 0

In [7]:
med['opioid'] = med.drugname.apply(lambda x: opioids(x))
med['nsaid'] = med.drugname.apply(lambda x: nsaids(x))

med = med[med.drugordercancelled == 'No']

In [8]:
cols_drop = ['medicationid','drugorderoffset','drugivadmixture','drugordercancelled','drughiclseqno',
            'routeadmin','loadingdose','prn','gtc']

med.drop(columns=cols_drop, inplace=True)
med = med[(med.opioid == 1) | (med.nsaid == 1)]

In [9]:
med.head()

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,drugstopoffset,opioid,nsaid
2,141168,1386,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2390,0,1
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,12622,1,0
33,141203,-339,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2414,0,1
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,4189,1,0
51,141227,-843,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,1744,0,1


In [13]:
mg = med.groupby(['patientunitstayid']).agg({'opioid': lambda x:sum(x), 'nsaid': lambda x:sum(x)})
mg[mg != 0] = 1

In [14]:
def op_find (x, arr):
    try:
        return arr['opioid'][x]
    except:
        return np.nan
    
def nsaid_find (x, arr):
    try:
        return arr['nsaid'][x]
    except:
        return np.nan

In [18]:
pts = pd.read_csv('eicu-collaborative-research-database-2.0/patient.csv.gz')
hos = pd.read_csv('eicu-collaborative-research-database-2.0/hospital.csv.gz')

In [19]:
pts['opioid'] = pts['patientunitstayid'].apply(lambda x: op_find(x, mg))
pts['nsaid'] = pts['patientunitstayid'].apply(lambda x: nsaid_find(x, mg))

pts = pd.merge(pts, hos, on=['hospitalid'], how = 'left')

In [20]:
pts = pts[pts.gender != 'Unknown']
pts = pts[pts.gender != 'Other']

In [13]:
pts['apacheadmissiondx'].fillna(value = 'N/A', inplace = True)
pts['hospitaladmitsource'].fillna(value = 'Unknown', inplace = True)

def age (x):
    if x == '> 89':
        return 90
    else:
        return int(x)

pts['age'].fillna(value = 0, inplace = True)
pts['age'] = pts['age'].apply(lambda x: age(x))

In [14]:
pts.opioid.fillna(0, inplace = True)
pts.nsaid.fillna(0, inplace = True)

In [15]:
pts.opioid.replace(1.0,True,inplace=True)
pts.opioid.replace(0.0,False,inplace=True)

pts.nsaid.replace(1.0,True,inplace=True)
pts.nsaid.replace(0.0,False,inplace=True)

In [16]:
pts['painmeds'] = pts.opioid|pts.nsaid

In [17]:
pts.head()

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,hospitaladmitoffset,hospitaladmitsource,hospitaldischargeyear,hospitaldischargetime24,hospitaldischargeoffset,hospitaldischargelocation,hospitaldischargestatus,unittype,unitadmittime24,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid,opioid,nsaid,numbedscategory,teachingstatus,region,painmeds
0,141168,128919,Female,70,Caucasian,59,91,"Rhythm disturbance (atrial, supraventricular)",152.4,15:54:00,0,Direct Admit,2015,03:50:00,3596,Death,Expired,Med-Surg ICU,15:54:00,Direct Admit,1,admit,84.3,85.8,03:50:00,3596,Death,Expired,002-34851,True,True,<100,f,Midwest,True
1,141178,128927,Female,52,Caucasian,60,83,N/A,162.6,08:56:00,-14,Emergency Department,2015,19:20:00,2050,Home,Alive,Med-Surg ICU,09:10:00,Emergency Department,1,admit,54.4,54.4,09:18:00,8,Step-Down Unit (SDU),Alive,002-33870,False,True,<100,f,Midwest,True
2,141179,128927,Female,52,Caucasian,60,83,N/A,162.6,08:56:00,-22,Emergency Department,2015,19:20:00,2042,Home,Alive,Med-Surg ICU,09:18:00,ICU to SDU,2,stepdown/other,NaN,60.4,19:20:00,2042,Home,Alive,002-33870,False,False,<100,f,Midwest,False
3,141194,128941,Male,68,Caucasian,73,92,"Sepsis, renal/UTI (including bladder)",180.3,18:18:40,-780,Floor,2015,23:30:00,12492,Home,Alive,CTICU,07:18:00,Floor,1,admit,73.9,76.7,15:31:00,4813,Floor,Alive,002-5276,True,True,>= 500,t,Midwest,True
4,141196,128943,Male,71,Caucasian,67,109,N/A,162.6,20:21:00,-99,Emergency Department,2015,17:00:00,5460,Home,Alive,Med-Surg ICU,22:00:00,ICU to SDU,2,stepdown/other,NaN,63.2,22:23:00,1463,Floor,Alive,002-37665,True,True,NaN,f,Midwest,True


In [ ]:
apv = pd.read_csv('eicu-collaborative-research-database-2.0/apachePredVar.csv.gz')
apv = apv[['patientunitstayid', 'bedcount']]
pts = pd.merge(pts, apv, on=['patientunitstayid'], how='left')

In [ ]:
# hospital odds ratios

pts_hosp = pts[['patientunitstayid','ethnicity','hospitalid','wardid','region','numbedscategory',
                'bedcount','teachingstatus','opioid','nsaid','painmeds']]
pts_hosp.head()

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='gender',y='painmeds',hue='ethnicity',data=pts, estimator=np.mean)
plt.xlabel('Patient Gender')
plt.ylabel('Proportion Receiving Pain Medications')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='gender',y='nsaid',hue='ethnicity',data=pts, estimator=np.mean)
plt.xlabel('Patient Gender')
plt.ylabel('Proportion Receiving NSAIDs')

In [ ]:
plt.figure(figsize=(14, 4.5))
sns.barplot(x='gender',y='opioid',hue='ethnicity',data=pts, estimator=np.mean)
plt.xlabel('Patient Gender')
plt.ylabel('Proportion Receiving Opioids')

In [ ]:
ct_eth = pd.crosstab(pts.painmeds, pts.ethnicity, margins= True)
ct_eth

In [ ]:
fobs = [14170, 1933, 99411, 4763, 1214, 6736]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

In [ ]:
ct_op = pd.crosstab(pts.opioid, pts.ethnicity, margins= True)
ct_op

In [ ]:
fobs = [10510,1487,75398,3175,989,5277]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

In [ ]:
ct_ns = pd.crosstab(pts.nsaid, pts.ethnicity, margins= True)
ct_ns

In [ ]:
fobs = [12165,1688,83939,4151,868,5634]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

In [ ]:
dxs = pd.read_csv('eicu-collaborative-research-database-2.0/diagnosis.csv.gz')

# adding icd9 codes
dxs = dxs.drop(columns=['diagnosisid', 'activeupondischarge', 'diagnosisoffset', 'diagnosisstring'])
dxs.dropna(inplace=True)

# merge
dxs = dxs.groupby('patientunitstayid')['icd9code'].apply(list).reset_index(name='icd9codes')
pts = pd.merge(pts, dxs, on=['patientunitstayid'], how='left')

In [ ]:
cpg = pd.read_csv('eicu-collaborative-research-database-2.0/carePlanGeneral.csv.gz')

# adding ordered protocols
cpg = cpg[cpg.cplgroup == 'Ordered Protocols']
cpg.drop(columns=['cplgeneralid', 'activeupondischarge', 'cplitemoffset', 'cplgroup'], inplace = True)
cpg.drop_duplicates(inplace=True)

# merge
cpg = cpg.groupby('patientunitstayid')['cplitemvalue'].apply(list).reset_index(name='orderedprotocols')
pts = pd.merge(pts, cpg, on=['patientunitstayid'], how = 'left')

In [ ]:
nsc = pd.read_csv('eicu-collaborative-research-database-2.0/nurseCharting.csv.gz')

# adding pain scores
nsc.drop(['nursingchartid', 'nursingchartoffset', 
          'nursingchartcelltypecat', 'nursingchartcelltypevallabel'], axis=1, inplace=True)
nsc = nsc[nsc['nursingchartcelltypevalname'] == 'Pain Score']

In [ ]:
valid = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]

nsc['nursingchartvalue'] = nsc['nursingchartvalue'].apply(lambda x: float(x))
nsc = nsc[nsc['nursingchartvalue'].isin(valid)]

nsc['nursingchartvalue'] = nsc['nursingchartvalue'].apply(lambda x: int(x))
nsc.drop(['nursingchartcelltypevalname'], axis=1, inplace=True)

In [ ]:
max_pain = nsc.groupby('patientunitstayid').max()
mean_pain = nsc.groupby('patientunitstayid').mean()

In [ ]:
def pain_val (x, arr):
    try:
        return round(arr['nursingchartvalue'][x])
    except:
        return np.nan

pts['max_pain'] = pts['patientunitstayid'].apply(lambda x: pain_val(x, max_pain))
pts['mean_pain'] = pts['patientunitstayid'].apply(lambda x: pain_val(x, mean_pain))

In [ ]:
# Two way T test for Native American no-analgesia vs all other Ethnicities no-analgesia PAIN SCORES
npts = pts[pts.painmeds == False]

natpts = npts[npts.ethnicity == 'Native American']
othpts = npts[npts.ethnicity != 'Native American']

print(stat.ttest_ind(natpts.mean_pain.dropna(), othpts.mean_pain.dropna()))

**Difference in pain scores between Native American no-analgesia vs all other Ethnicities no-analgesia not significant**

In [ ]:
# pts['native'] = pts.ethnicity

# pts.native.replace('African American', 'Non-native', inplace=True)
# pts.native.replace('Asian', 'Non-native', inplace=True)
# pts.native.replace('Caucasian', 'Non-native', inplace=True)
# pts.native.replace('Hispanic', 'Non-native', inplace=True)
# pts.native.replace('Other/Unknown', 'Non-native', inplace=True)
# pts.native.replace('Native American', 'Native', inplace=True)

In [ ]:
ct_eth = pd.crosstab(pts.painmeds, pts.ethnicity, margins= True)
ct_eth

In [ ]:
fobs = [14550, 1953, 100655, 4770, 1218, 6796]
ftots = [21303, 3270, 155273, 7462, 1700, 9523]
fexp = np.multiply(np.sum(fobs)/np.sum(ftots),ftots)
stat.chisquare(f_obs = fobs, f_exp = fexp)

_____
#### Temporal Analysis

In [18]:
med.head()

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,drugstopoffset,opioid,nsaid
2,141168,1386,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2390,0,1
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,12622,1,0
33,141203,-339,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,2414,0,1
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,4189,1,0
51,141227,-843,ASPIRIN EC 81 MG PO TBEC,81 3,Daily,1744,0,1


In [19]:
med['duration_min'] = med.drugstopoffset - med.drugstartoffset
med.drop(columns=['drugstopoffset'], inplace=True)

In [20]:
med.dropna(inplace=True)

In [21]:
opi = med[med.opioid==1]

In [22]:
opi.head()

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,opioid,nsaid,duration_min
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,1,0,10513
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,1,0,2316
62,141229,-100,MORPHINE INJ,4 3,Q30 Min PRN,1,0,3728
94,141244,-11713,MORPHINE INJ,4 3,Q30 Min PRN,1,0,196
104,141260,30,OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS,1 5002,Q6H PRN,1,1,1969


In [23]:
opi = opi[opi.frequency != 'Pyxis']

In [24]:
dict(opi.frequency.value_counts())

{'Q4H PRN': 23770,
 'Q4H': 22061,
 'Once': 14489,
 'Q2H': 13443,
 'Q2H PRN': 13258,
 'q 4 hour PRN': 10318,
 'Q1H': 9806,
 'Once X1': 9668,
 'Q1H PRN': 9641,
 'Q6H PRN': 9310,
 'Q5 Min PRN': 9227,
 '.STK-MED': 9050,
 'Q3H': 7172,
 'q4hr': 6480,
 'Q3H PRN': 5908,
 'Q4HPRN': 5564,
 'ZPYXISVEND': 5519,
 'Q15MINPRN': 4911,
 '1XONLY': 4892,
 'Every 4 hours PRN': 4741,
 'PACU USE ONLY': 4011,
 'ONETIME': 3916,
 'q2hr': 3706,
 'ONCE X1': 3578,
 'Q3HPRN': 3391,
 'q 2 hour PRN': 3303,
 'Q2HPRN': 3220,
 'Every 5 min PRN': 2967,
 'Q6H': 2743,
 'ONE': 2693,
 'Q10 Min PRN': 2671,
 'PRN': 2563,
 'CONTINUOUS': 2533,
 'q 1 hour PRN': 2293,
 'TITRATE': 2141,
 'q1hr': 1991,
 'Q5MIN X5': 1840,
 'PYX1TIME': 1804,
 'Q1HPRN': 1664,
 'Q30MIN': 1624,
 'POSTOP': 1583,
 'Q15 Min PRN': 1570,
 'Every 2 hours PRN': 1570,
 'Q6HPRN': 1456,
 'Q4HP': 1450,
 'q5min H4': 1368,
 'PACU USE ONLY X1': 1164,
 'q 6 hour PRN': 1153,
 'q5min': 1138,
 'q15minprn INDEF': 1102,
 'ONCALL': 1076,
 'Every 3 hours PRN': 1061,
 'Q2MIN 

In [25]:
dict(opi.dosage.value_counts())

{'2 mg': 25974,
 '1 Tab': 17256,
 '50 mcg': 16217,
 '2 MG': 15234,
 '4 mg': 13721,
 '25 mcg': 8907,
 '5 mg': 8134,
 '1 mg': 8127,
 '100 MCG': 7453,
 '1 MG': 7361,
 '25 MCG': 6628,
 '50 MCG': 6175,
 '2 Tab': 6122,
 '0.5 mg': 5907,
 '4 MG': 5667,
 '5 MG': 5494,
 '1-2 5002': 5430,
 '1 TAB': 4623,
 '1 tablet': 4499,
 '10 mg': 4378,
 '100 mcg': 4312,
 '25 8': 4028,
 '5-10 mg': 3790,
 '10 MG': 3417,
 '0.5 MG': 3328,
 '0.2 3': 3201,
 '6 mg': 3180,
 '12.5-50 mcg': 3016,
 '2 ML': 3000,
 '50 MG': 2825,
 '1-2 TAB': 2628,
 '100 ML': 2313,
 '50 mg': 2266,
 '1 ML': 2032,
 '1 5002': 2021,
 '2 3': 1986,
 '0.2-0.8 mg': 1830,
 '1 tab(s)': 1823,
 'MG': 1808,
 '1-4 3': 1750,
 '1 Tablet': 1720,
 '25-50 MCG': 1691,
 '1-2 tablet': 1653,
 '25-50 8': 1581,
 '8 mg': 1561,
 '25-50 mcg': 1552,
 '1 3': 1514,
 '1000 MCG': 1443,
 '1-2 MG': 1422,
 '1 TABLET': 1382,
 '50 3': 1367,
 '50-100 MCG': 1335,
 'Manual Charge': 1328,
 '2-4 mg': 1286,
 '25 mg': 1217,
 '0.2-0.4 mg': 1200,
 '0.2-0.5 3': 1171,
 '5-15 mg': 1166,
 '

In [26]:
import re

In [27]:
t = '25-200 mcg/hr'

In [28]:
def cases (x):
    
    try:
        if 'mcg/hr' in x.lower():
            regex = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            regex = [float(i) for i in regex]

            if len(regex) > 1:
                return str(regex[-1]/1000) + ' hr'
            else:
                return str(regex[0]/1000) + ' hr'
    
        elif 'mg' in x.lower():
            regex = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            regex = [float(i) for i in regex]

            if len(regex) > 1:
                return regex[-1]
            else:
                return regex[0]

        elif 'mcg' in x.lower():
            regex = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            regex = [float(i) for i in regex]

            if len(regex) > 1:
                return regex[-1]/1000
            else:
                return regex[0]/1000
        else:
            return x.lower()
    except:
        return x.lower()

In [29]:
cases(t)

'0.2 hr'

In [30]:
opi['dos_mg'] = opi.dosage.apply(lambda x: cases(x))

In [31]:
opi

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,opioid,nsaid,duration_min,dos_mg
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,1,0,10513,0.4 3
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,1,0,2316,2 3
62,141229,-100,MORPHINE INJ,4 3,Q30 Min PRN,1,0,3728,4 3
94,141244,-11713,MORPHINE INJ,4 3,Q30 Min PRN,1,0,196,4 3
104,141260,30,OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS,1 5002,Q6H PRN,1,1,1969,1 5002
...,...,...,...,...,...,...,...,...,...
7296514,3350616,-172,OXYCODONE/APAP 5MG/325MG,2 TAB,Q4HRSPRN,1,0,172,2 tab
7296515,3350616,-172,OXYCODONE/APAP 5MG/325MG,1 TAB,Q4HRSPRN,1,0,172,1 tab
7298742,3351717,-30,OXYCODONE,5 MG,Q6HRSPRN,1,0,30,5
7298745,3351717,-29,OXYCODONE,10 MG,Q6HRSPRN,1,0,29,10


In [32]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [33]:
opi.dos_mg.value_counts()

2.0       43909
0.05      28651
1 tab     21957
4.0       21806
1.0       18168
          ...  
400.0         1
14.0          1
300.0         1
75 3          1
0.2 hr        1
Name: dos_mg, Length: 120, dtype: int64

In [34]:
count = 0
drugnames = []

for index, row in opi.iterrows():
    if type(row.dos_mg) == str:
        if 'tab' in row.dos_mg:
            drugnames.append(row.drugname)

In [35]:
set(drugnames)

{'HYDROCODONE 5 MG-ACETAMINOPHEN 325 MG TABLET',
 'HYDROCODONE-ACETAMINOPHEN 10-325 MG PO TABS',
 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS',
 'HYDROCODONE/APAP 5MG/325MG',
 'HYDROcodone-acetaminophen',
 'NORCO',
 'NORCO 5/325 TAB',
 'Norco 5 mg-325 mg',
 'OXYCODONE HCL 5 MG PO TABS',
 'OXYCODONE-ACETAMIN 5-325 MG',
 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS',
 'OXYCODONE/APAP 5MG/325MG',
 'PERCOCET',
 'PERCOCET 5/325 TAB',
 'PERCOCET 5MG/325MG',
 'TRAMADOL HCL 50 MG PO TABS',
 'acetaminophen-oxyCODONE 325 mg-5 mg oral tablet',
 'oxyCODONE-acetaminophen',
 'oxycodone-acetaminophen 5-325 mg po tabs',
 'traMADol 50 MG TAB'}

In [36]:
#morphine equivalent/day/stay

In [37]:
opi.drugname.replace('HYDROCODONE 5 MG-ACETAMINOPHEN 325 MG TABLET', 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('HYDROCODONE/APAP 5MG/325MG', 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('HYDROcodone-acetaminophen', 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('NORCO', 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('NORCO 5/325 TAB', 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('Norco 5 mg-325 mg', 'HYDROCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)

opi.drugname.replace('OXYCODONE-ACETAMIN 5-325 MG', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('OXYCODONE/APAP 5MG/325MG', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('PERCOCET', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('PERCOCET 5/325 TAB', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('PERCOCET 5MG/325MG', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)

opi.drugname.replace('acetaminophen-oxyCODONE 325 mg-5 mg oral tablet', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('oxyCODONE-acetaminophen', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)
opi.drugname.replace('oxycodone-acetaminophen 5-325 mg po tabs', 'OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS', inplace = True)

opi.drugname.replace('traMADol 50 MG TAB', 'TRAMADOL HCL 50 MG PO TABS', inplace = True)

In [38]:
opi.head()

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,opioid,nsaid,duration_min,dos_mg
15,141194,2109,1 ML - HYDROMORPHONE HCL 1 MG/ML IJ SOLN,0.4 3,Q3H PRN,1,0,10513,0.4 3
37,141203,1873,MORPHINE INJ,2 3,Q15 Min PRN,1,0,2316,2 3
62,141229,-100,MORPHINE INJ,4 3,Q30 Min PRN,1,0,3728,4 3
94,141244,-11713,MORPHINE INJ,4 3,Q30 Min PRN,1,0,196,4 3
104,141260,30,OXYCODONE-ACETAMINOPHEN 5-325 MG PO TABS,1 5002,Q6H PRN,1,1,1969,1 5002


In [ ]:
# ignore the second number - doesn't really make sense

In [42]:
opi.patientunitstayid.value_counts()

963286     104
3049689     74
3121702     54
3077784     54
3105752     53
          ... 
2521021      1
3035066      1
933812       1
1064820      1
2405863      1
Name: patientunitstayid, Length: 89293, dtype: int64

In [45]:
opi[opi['patientunitstayid'] == 963286]

,patientunitstayid,drugstartoffset,drugname,dosage,frequency,opioid,nsaid,duration_min,dos_mg
353652,963286,662,HYDROmorphone,Manual Charge,Once X1,1,0,0,manual charge
2366775,963286,11260,HYDROmorphone,1 mg,Once,1,0,0,1
2366792,963286,-351,HYDROmorphone,2 mg,Once,1,0,0,2
2366793,963286,11462,HYDROmorphone,1 mg,Once,1,0,0,1
2366795,963286,-561,HYDROmorphone,1 mg,Once,1,0,0,1
...,...,...,...,...,...,...,...,...,...
5975867,963286,10985,fentaNYL,250 mcg,Once,1,0,0,0.25
5975868,963286,10923,fentaNYL,100 mcg,Once,1,0,0,0.1
5975869,963286,10964,fentaNYL,250 mcg,Once,1,0,0,0.25
5975870,963286,10976,fentaNYL,250 mcg,Once,1,0,0,0.25
